# ДЗ 4

- Студент: Алексей Ярошенко
- Email на портале: aleksey.yaroshenko@gmail.com
- https://docs.google.com/document/d/1z35Z-xctrrYO8r6KD10w8daKCLEMkCBQW9iNk2nEskk

In [1729]:
from collections import Counter, defaultdict
import re
import random
from copy import copy

import pandas as pd
import numpy as np
from nltk.tokenize import RegexpTokenizer
from nltk import everygrams
from tqdm import tqdm

## 1. Реализуйте базовый частотный метод по Шерлоку Холмсу

- подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);
- возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе совсем вряд ли сработает), зашифруйте их посредством случайной перестановки символов;
- расшифруйте их таким частотным методом.


In [1625]:
tokenizer = RegexpTokenizer(r'\w+')
abc = ' абвгдежзийклмнопрстуфхцчшщъыьэюя'

def tokenize(text, tokenizer=tokenizer):
    text = ''.join([c for c in text if c.lower() in abc])
    return ' '.join(tokenizer.tokenize(text.lower()))


def get_freqs(text, min_freq=0, n_gram=1):
    freqs = dict()
    if n_gram > 1:
        text = [''.join(ngram) for ngram in everygrams(text, min_len=n_gram, max_len=n_gram)]
    for key, value in Counter(text).items():
        if value/len(text) > min_freq:
            freqs[key] = value/len(text)
    return freqs


def generate_mapping(freqs):
    original = list(freqs.keys())
    replacements = np.random.choice(original, replace=False, size=len(freqs))
    mapping = dict()
    for original_char, replacement_char in zip(original, replacements):
        mapping[original_char] = replacement_char
    return mapping


def apply_mapping(text, mapping):
    return ''.join([mapping.get(c, 'ь') for c in text])

# Здесь мы находим ближайший по частотности символ.
# Пробовал использовать просто ранги, но качество хуже.
def get_reverse_mapping(corpus_freqs, text_freqs):
    corpus_freqs_sorted = sorted(corpus_freqs.items(), key=lambda x: x[1], reverse=True)
    text_freqs_sorted = sorted(text_freqs.items(), key=lambda x: x[1], reverse=True)
    
    reverse_mapping = dict()
    for text_char, text_freq in text_freqs_sorted:
        min_diff = 1
        best_char = None
        for corpus_char, corpus_freq in corpus_freqs_sorted:
            diff = abs(corpus_freq - text_freq)
            if diff < min_diff:
                best_char = corpus_char
                min_diff = diff

        reverse_mapping[text_char] = best_char
        corpus_freqs_sorted = [(char, freq) for char, freq in corpus_freqs_sorted if char !=best_char]
        
    return reverse_mapping


def char_accuracy(text1, text2):
    assert len(text1) == len(text2)
    right_chars = 0
    for c1, c2 in zip(text1, text2):
        right_chars += int(c1 == c2)
    return right_chars / len(text1)

In [1693]:
np.random.seed(42)

with open('corpora/AnnaKarenina.txt' ,'r') as file:
    corpus1 = file.readlines()
    
with open('corpora/WarAndPeace.txt' ,'r') as file:
    corpus2 = file.readlines()
    
corpus = corpus1 + corpus2

tokenized_corpus = tokenize(' '.join(corpus))
corpus_freqs = get_freqs(tokenized_corpus)
mapping = generate_mapping(corpus_freqs)

In [1694]:
with open('corpora/text.txt' ,'r') as file:
    text = file.readlines()

tokenized_text = tokenize(' '.join(text))
encoded_text = apply_mapping(tokenized_text, mapping)
text_freqs = get_freqs(encoded_text)
tokenized_text

'в течение многих часов шерлок холмс сидел согнувшись над стеклянной пробиркой в которой варилось чтото на редкость вонючее голова его была опущена на грудь и он казался мне похожим на странную товцую птицу с тусклыми серыми перьями и черным хохолком итак уотсон сказал он внезапно вы не собираетесь вкладывать свои сбережения в южноафриканские ценные бумаги я вздрогнул от удивления как ни привык я к необычайным способностям холмса это внезапное вторжение в самые тайные мои мысли было совершенно необъяснимым как черт возьми вы об этом узнали спросил я он повернулся на стуле держа в руке дымящуюся пробирку и его глубоко сидящие глаза радостно заблистали признайтесь уотсон что вы совершенно сбиты с толку сказал он признаюсь мне следовало бы заставить вас написать об этом на листочке бумаги и подписаться почему потому что через пять минут вы скажете что все это необычайно просто'

In [1695]:
encoded_text

'пумзкзвчзулвшючхукщршпуфздншьухшнлрурчызнуршювипфчр увщыурмзьнэввшбугдшсчдьшбупуьшмшдшбупщдчншр укмшмшувщудзыьшрм упшвокззуюшншпщузюшусенщушгитзвщувщуюдиы учушвуьщащнрэулвзугшхшжчлувщурмдщввиоумшпъиоугмчъиурумирьнелчурзделчугзд элчучукздвелухшхшньшлучмщьуишмршвурьщащнушвупвзащгвшупеувзуршсчдщзмзр упьнщыепщм урпшчурсздзжзвчэупуожвшщйдчьщврьчзуъзввезусилщючуэупаыдшювинушмуиычпнзвчэуьщьувчугдчпеьуэуьувзшсекщбвелургшршсвшрмэлухшнлрщуямшупвзащгвшзупмшджзвчзупурщлезумщбвезулшчулернчусеншуршпздфзввшувзшсцэрвчлелуьщьукздмупша лчупеушсуямшлуиавщнчургдшрчнуэушвугшпздвинрэувщурминзуызджщупудиьзуыелэтиорэугдшсчдьиучузюшуюнисшьшурчыэтчзуюнщащудщышрмвшуащснчрмщнчугдчавщбмзр уишмршвукмшупеуршпздфзввшурсчмеурумшньиурьщащнушвугдчавщор улвзурнзышпщншусеуащрмщпчм упщрувщгчрщм ушсуямшлувщунчрмшкьзусилщючучугшыгчрщм рэугшкзлиугшмшлиукмшукздзаугэм улчвимупеурьщжзмзукмшупрзуямшувзшсекщбвшугдшрмш'

In [1696]:
reverse_mapping = get_reverse_mapping(corpus_freqs, text_freqs)
decoded_text = apply_mapping(encoded_text, reverse_mapping)
decoded_text

'р лнынасн даожсх ытиор энвком хокди исчнк иожаурэсиб атч илнмкгааош явоьсвмош р моловош ртвскоиб ылоло ат внчмоилб роаюынн жокорт нжо ьпкт ояущнат ат жвучб с оа мтзткиг дан яохойсд ат илвтааую лорфую ялсфу и луимкпдс инвпдс янвбгдс с ынвапд хохокмод слтм уолиоа имтзтк оа ранзтяао рп ан иоьсвтнлниб рмктчпртлб ирос иьнвнйнасг р юйаотъвсмтаимсн фнаапн ьудтжс г рзчвожаук ол учсркнасг мтм ас явсрпм г м аноьпытшапд ияоиоьаоилгд хокдит цло ранзтяаон рловйнасн р итдпн лтшапн дос дпикс ьпко иорнвэнаао аноьегиасдпд мтм ынвл розбдс рп оь цлод узаткс иявоиск г оа яорнваукиг ат илукн чнвйт р вумн чпдгщуюиг явоьсвму с нжо жкуьомо исчгщсн жктзт втчоилао зтьксилткс явсзатшлниб уолиоа ыло рп иорнвэнаао иьслп и локму имтзтк оа явсзатюиб дан икнчортко ьп зтилтрслб рти атяситлб оь цлод ат ксилоымн ьудтжс с яочяситлбиг яоынду яолоду ыло ынвнз яглб дсаул рп имтйнлн ыло рин цло аноьпытшао явоило'

In [1697]:
char_accuracy(tokenized_text, decoded_text)

0.31186440677966104

В общем, так себе разультат :)

## 2. Базовый частотный метод с биграммами


Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы. Но и Шерлок Холмс был не так уж прост: после буквы E, которая действительно выделяется частотой, дальше он анализировал уже конкретные слова и пытался угадать, какими они могли бы быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что давайте просто сделаем следующий логический шаг:

- подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;
- проведите тестирование аналогично п.1, но при помощи биграмм.

In [1698]:
corpus_freqs_bigram = get_freqs(tokenized_corpus, n_gram=2)
text_freqs_bigram = get_freqs(encoded_text, n_gram=2)

corpus_freqs_sorted = sorted(corpus_freqs_bigram.items(), key=lambda x: x[1], reverse=True)
text_freqs_sorted = sorted(text_freqs_bigram.items(), key=lambda x: x[1], reverse=True)

In [1699]:
# Здесь мы проходимся по биграмам, начиная от самых частотных, 
# на каждом шаге учитывая раскодированные ранее символы

def get_reverse_mapping_bigram(corpus_freqs, text_freqs, init_mapping=None):
    corpus_freqs_sorted = sorted(corpus_freqs.items(), key=lambda x: x[1], reverse=True)
    text_freqs_sorted = sorted(text_freqs.items(), key=lambda x: x[1], reverse=True)
    
    if init_mapping is None:
        init_mapping = []
    reverse_mapping = {k: v for k, v in init_mapping}
    
    for i, (text_bigram, text_freq) in enumerate(text_freqs_sorted):

        filtred_freqs = copy(corpus_freqs_sorted)
        
        if text_bigram[0] in reverse_mapping.keys():
            filtred_freqs = [(bigram, freq) for bigram, freq in filtred_freqs if bigram[0] == reverse_mapping[text_bigram[0]]]
            
        if text_bigram[1] in reverse_mapping.keys():
            filtred_freqs = [(bigram, freq) for bigram, freq in filtred_freqs if bigram[1] == reverse_mapping[text_bigram[1]]]
              
        min_diff = 1
        best_bigram = None
        for bigram, freq in filtred_freqs:
            diff = abs(freq - text_freq)
            if diff < min_diff:
                best_bigram = bigram
                min_diff = diff
                
        if text_bigram[0] not in reverse_mapping.keys():
            reverse_mapping[text_bigram[0]] = best_bigram[0]
            
        if text_bigram[1] not in reverse_mapping.keys():
            reverse_mapping[text_bigram[1]] = best_bigram[1]

        
    return reverse_mapping

In [1700]:
tokenized_text

'в течение многих часов шерлок холмс сидел согнувшись над стеклянной пробиркой в которой варилось чтото на редкость вонючее голова его была опущена на грудь и он казался мне похожим на странную товцую птицу с тусклыми серыми перьями и черным хохолком итак уотсон сказал он внезапно вы не собираетесь вкладывать свои сбережения в южноафриканские ценные бумаги я вздрогнул от удивления как ни привык я к необычайным способностям холмса это внезапное вторжение в самые тайные мои мысли было совершенно необъяснимым как черт возьми вы об этом узнали спросил я он повернулся на стуле держа в руке дымящуюся пробирку и его глубоко сидящие глаза радостно заблистали признайтесь уотсон что вы совершенно сбиты с толку сказал он признаюсь мне следовало бы заставить вас написать об этом на листочке бумаги и подписаться почему потому что через пять минут вы скажете что все это необычайно просто'

In [1701]:
encoded_text

'пумзкзвчзулвшючхукщршпуфздншьухшнлрурчызнуршювипфчр увщыурмзьнэввшбугдшсчдьшбупуьшмшдшбупщдчншр укмшмшувщудзыьшрм упшвокззуюшншпщузюшусенщушгитзвщувщуюдиы учушвуьщащнрэулвзугшхшжчлувщурмдщввиоумшпъиоугмчъиурумирьнелчурзделчугзд элчучукздвелухшхшньшлучмщьуишмршвурьщащнушвупвзащгвшупеувзуршсчдщзмзр упьнщыепщм урпшчурсздзжзвчэупуожвшщйдчьщврьчзуъзввезусилщючуэупаыдшювинушмуиычпнзвчэуьщьувчугдчпеьуэуьувзшсекщбвелургшршсвшрмэлухшнлрщуямшупвзащгвшзупмшджзвчзупурщлезумщбвезулшчулернчусеншуршпздфзввшувзшсцэрвчлелуьщьукздмупша лчупеушсуямшлуиавщнчургдшрчнуэушвугшпздвинрэувщурминзуызджщупудиьзуыелэтиорэугдшсчдьиучузюшуюнисшьшурчыэтчзуюнщащудщышрмвшуащснчрмщнчугдчавщбмзр уишмршвукмшупеуршпздфзввшурсчмеурумшньиурьщащнушвугдчавщор улвзурнзышпщншусеуащрмщпчм упщрувщгчрщм ушсуямшлувщунчрмшкьзусилщючучугшыгчрщм рэугшкзлиугшмшлиукмшукздзаугэм улчвимупеурьщжзмзукмшупрзуямшувзшсекщбвшугдшрмш'

In [1702]:
reverse_mapping_bigram = get_reverse_mapping_bigram(corpus_freqs_bigram, text_freqs_bigram)
decoded_text = apply_mapping(encoded_text, reverse_mapping_bigram)
decoded_text

' оатнтнттонн гтвона   оитон ков нн о тстно  гн  ит  онасо аткнннн но о кток но ок а о но аотн   она а онаоотск  а о  нлнттог н  аотг окенао   ятнаонаого с ото нокакан ноннто  в итнонао аоанн лоа  х ло атх о оа  кненто тоенто то ннтотонтоненов в нк нотаако  а  но какано но нтка н о еонто  ктоатат  о кнасе аа о   то ктотитнтно олин ажоткан кттохтннеток нагтоно ксо гн но ао ст нтнтнокаконто от еконоконт кенаннено     кн  аннов нн аога о нтка н то а оитнтто о анетоааннетон тоне нтокен о   тоитнн онт кнн нтненокаконтоао  к нто ео кога но кнанто  о  тноно но   тон н нонао а нтостоиао оо ктосення л но о кток ототг огн к к о тсняттогнакаооас  ан окакнт аанто откнанат  о  а  нона о ео   тоитнн о ктаео оа нк о какано но откнал  оннто нтс  ан океока аа та о а она т аа о кога нонаонт а нкток нагтото  с т аа  но  нтн о  а н она онтотко на онтн ао ео каитатона о  тога онт кенанн о о  а '

In [1703]:
char_accuracy(tokenized_text, decoded_text)

0.16497175141242937

Стало хуже, что неудивительно. Биграмм больше, попасть по частоте в правильные - сложнее. Нужен текст сильно больше.

Попробуем добавить 1-2 раскодированных с помощью посимвольного метода символа как инициализацию.

In [1704]:
init_mapping = list(reverse_mapping.items())[:2]
init_mapping

[('у', ' '), ('ш', 'о')]

In [1705]:
reverse_mapping_bigram = get_reverse_mapping_bigram(corpus_freqs_bigram, text_freqs_bigram, init_mapping=init_mapping)
decoded_text = apply_mapping(encoded_text, reverse_mapping_bigram)
decoded_text

'с такаоаа ьооеал кнсос  алиоо лоиьс саыаи соеорс ася оны стаоиьооо  слоналоо  с оотоло  снлаиося ктото он лаыоостя сооукаа еоиосн аео нтин осркаон он елрыя а оо ононись ьоа соло аь он стлноору тосару стаар с трсоитьа салтьа саляььа а калоть лолоиооь атно ротсоо сонони оо соаонсоо ст оа соналнатася соинытснтя ссоа снала аоаь с у ооньлаоносоаа ааоота нрьнеа ь соылоеори от рыасиаоаь оно оа сласто ь о оаонткн оть ссосоноостьь лоиьсн ето соаонсооа стол аоаа с сньта тн ота ьоа ьтсиа нтио сосал аооо оаонььсоаьть оно калт соояьа ст он етоь роониа сслосаи ь оо сосалорись он стриа ыал н с лроа ытьькрусь слоналор а аео еирнооо саыькаа еинон лныостоо онниастниа слаоон тася ротсоо кто ст сосал аооо снатт с тоиор сонони оо слаоонуся ьоа сиаыоснио нт онстнсатя снс онсаснтя он етоь он иастокоа нрьнеа а соысаснтясь сокаьр сотоьр кто калао сьтя ьаорт ст сон ата кто сса ето оаонткн оо слосто'

In [1706]:
char_accuracy(tokenized_text, decoded_text)

0.36045197740112994

Метрики выросли, но прочесть это все так же невозможно :)

## 3. Метод на основе MCMC-семплирования

Но и это ещё не всё: биграммы скорее всего тоже далеко не всегда работают. Основная часть задания — в том, как можно их улучшить:
- предложите метод обучения перестановки символов в этом задании, основанный на MCMC-сэмплировании, но по-прежнему работающий на основе статистики биграмм;
- реализуйте и протестируйте его, убедитесь, что результаты улучшились.


**Метод**

Текст, разбитый на биграммы - это, по сути, марковская цепь. Частота биграмм - вероятность перехода по цепи.

Для обучения перестановки будем считать вероятность порождения именно такого текста как произведение вероятностей всех биграмм, в него входящих. 

Всего перестановок очень много, поэтому сделаем жадный алгоритм, который использует идею MCMC-семплирования.

Алгоритм:

0) Инициализируем перестановки, восстанавливаем текст и считаем на нем $p_{current}$ 

1) Меняем местами пару букв для перестановки

2) Восстанавливаем текст с новой перестановкой и считаем на нем $p_{proposal}$  

3) Принимаем новую перестановку с "вероятностью" $p_{accept} = \frac{p_{proposal}}{p_{current}}$ 

4) Возвращаемся к пункту 1

Мы можем уйти немного не туда и застрять не в том максимуме, поэтому будем делать много попыток и брать лучшую.

In [1758]:
def get_freqs_smooth(text, min_freq=0, n_gram=2):
    freqs = dict()
    vocab_len = len(set(text))**n_gram
    if n_gram > 1:
        text = [''.join(ngram) for ngram in everygrams(text, min_len=n_gram, max_len=n_gram)]
    for key, value in Counter(text).items():
        freqs[key] = (value + 1) / (len(text) + vocab_len) # Сглаживаем, чтобы не было нулей
    return freqs

def get_text_proba(text, mapping, freqs, n_gram=2):
    decoded_text = apply_mapping(text, mapping)
    log_proba = 0
    for i in range(len(decoded_text) - n_gram):
        bigram = decoded_text[i: i + n_gram]
        bigram_proba = freqs.get(bigram)
        if bigram_proba is None:
            bigram_proba = 1 / (len(text) + len(abc)**n_gram) # Сглаживаем, чтобы не было нулей
            
        log_proba += np.log(bigram_proba)
    return log_proba
        
def get_reverse_mapping_mcmc(encoded_text, abc_encoded, abc_corpus, freqs_corpus, n_iters=10000, n_trials=10, n_gram=2):

    accept_count = 0
    best_mapping = None
    all_mappings = []
    best_log_likekihood = -np.inf

    for trial in tqdm(range(n_trials), leave=False, position=0, total=n_trials):

        abc_encoded = list(abc_encoded)
        abc_iter = list(abc_corpus)
        reverse_mapping = {k: v for k, v in zip(abc_encoded, abc_iter[:len(abc_encoded)])}
        log_proba_current = get_text_proba(encoded_text, reverse_mapping, freqs_corpus, n_gram=n_gram)

        for i in range(n_iters):
            abc_proposal = abc_iter[:]
            idx1, idx2 = np.random.choice(len(abc_proposal), replace=False, size=2)
            abc_proposal[idx1], abc_proposal[idx2] = abc_proposal[idx2], abc_proposal[idx1]
            reverse_mapping_proposal = {k: v for k, v in zip(abc_encoded, abc_proposal[:len(abc_encoded)])}
            log_proba_proposal = get_text_proba(encoded_text, reverse_mapping_proposal, freqs_corpus, n_gram=n_gram)

            p_accept = np.exp(log_proba_proposal - log_proba_current)

            if p_accept > np.random.rand():
                accept_count += 1
                abc_iter = abc_proposal
                log_proba_current = log_proba_proposal
                reverse_mapping = reverse_mapping_proposal

        if log_proba_current > best_log_likekihood:
            best_log_likekihood = log_proba_current
            best_mapping = reverse_mapping
            
        all_mappings.append(reverse_mapping)


    print(f'Best likelihood: {best_log_likekihood}')        
    print(f'Accept raito: {accept_count / (n_iters * n_trials)}')
    return best_mapping, all_mappings

In [1740]:
freqs_corpus = get_freqs_smooth(tokenized_corpus, n_gram=2)

In [1576]:
best_reverse_mapping, _ = get_reverse_mapping_mcmc(
    encoded_text, 
    abc_encoded=abc, 
    abc_corpus=abc,
    freqs_corpus=freqs_corpus
)

Trial 0 best likelihood: -5649.4222552997235
Trial 1 best likelihood: -4983.6918053880245
Trial 2 best likelihood: -5726.083384780235
Trial 3 best likelihood: -4986.237995189518
Trial 4 best likelihood: -5661.413882794688
Trial 5 best likelihood: -5033.768741636718
Trial 6 best likelihood: -4982.122522947728
Trial 7 best likelihood: -4983.6918053880245
Trial 8 best likelihood: -5591.465892426024
Trial 9 best likelihood: -5425.1575527392115
Best likelihood: -4982.122522947728
Accept raito: 0.01232


In [1577]:
apply_mapping(encoded_text, best_reverse_mapping)

'в течение многих часов шерлок холмс сидел согнувшись над стеклянной пробиркой в которой варилось чтото на редкость вонючее голова его была опуъена на грудь и он казался мне похожим на странную товцую птицу с тусклыми серыми перьями и черным хохолком итак уотсон сказал он внезапно вы не собираетесь вкладывать свои сбережения в южноафриканские ценные бумаги я вздрогнул от удивления как ни привык я к необычайным способностям холмса это внезапное вторжение в самые тайные мои мысли было совершенно необщяснимым как черт возьми вы об этом узнали спросил я он повернулся на стуле держа в руке дымяъуюся пробирку и его глубоко сидяъие глаза радостно заблистали признайтесь уотсон что вы совершенно сбиты с толку сказал он признаюсь мне следовало бы заставить вас написать об этом на листочке бумаги и подписаться почему потому что через пять минут вы скажете что все это необычайно просто'

Получилось :)

## 4. Расшифруйте сообщение

In [ ]:
message = "←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏"

In [1741]:
corpus_freqs_sorted = sorted(corpus_freqs.items(), key=lambda x: x[1], reverse=True)
message_freqs_sorted = sorted(message_freqs.items(), key=lambda x: x[1], reverse=True)

abc_corpus = ''.join([c for c, _ in corpus_freqs_sorted])
abc_message = ''.join([c for c, _ in message_freqs_sorted])
abc_corpus, abc_message

(' оеанитслвркдмупяьгыбзчжйшхюэцщфъ', '↹←⇛↟⇒↝⇴↨⇠⇯↷⇌⇊⇞⇈⇷↤↳↾↙⇙↲↞⇆⇰⇸↘⇏')

In [1742]:
freqs_corpus = get_freqs_smooth(tokenized_corpus, n_gram=2)

In [1747]:
best_reverse_mapping, _ = get_reverse_mapping_mcmc(
    message, 
    abc_encoded=abc_message, 
    abc_corpus=abc_corpus,
    freqs_corpus=freqs_corpus,
    n_iters=10000,
    n_trials=50,
)

Best likelihood: -1231.2547551943435
Accept raito: 0.042858


In [1748]:
apply_mapping(message, best_reverse_mapping)

'если вы вимите нордальный или почти нордальный текст у этого соожшения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный жалл за послемнее четвертое замание курса ботя конечно я ничего не ожешаю'

Успех :)

In [1753]:
original_message = "если вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать скорее всего вы все сделали правильно и получите максимальный балл за последнее четвертое задание курса хотя конечно я ничего не обещаю"
encoded_message = apply_mapping(message, best_reverse_mapping)
char_accuracy(original_message, encoded_message)

0.9391304347826087

## 5. A что если от биграмм перейти к триграммам

Бонус: а что если от биграмм перейти к триграммам (тройкам букв) или даже больше? Улучшатся ли результаты? Когда улучшатся, а когда нет? Чтобы ответить на этот вопрос эмпирически, уже может понадобиться погенерировать много тестовых перестановок и последить за метриками, глазами может быть и не видно.

**1. Попробуем раскодировать сообщение из пункта 4 триграммами**

In [1763]:
corpus_freqs_sorted = sorted(corpus_freqs.items(), key=lambda x: x[1], reverse=True)
message_freqs_sorted = sorted(message_freqs.items(), key=lambda x: x[1], reverse=True)

abc_corpus = ''.join([c for c, _ in corpus_freqs_sorted])
abc_message = ''.join([c for c, _ in message_freqs_sorted])
abc_corpus, abc_message

(' оеанитслвркдмупяьгыбзчжйшхюэцщфъ', '↹←⇛↟⇒↝⇴↨⇠⇯↷⇌⇊⇞⇈⇷↤↳↾↙⇙↲↞⇆⇰⇸↘⇏')

In [1764]:
freqs_corpus_trigramm = get_freqs_smooth(tokenized_corpus, n_gram=3)

In [1769]:
best_reverse_mapping, _ = get_reverse_mapping_mcmc(
    message, 
    abc_encoded=abc_message, 
    abc_corpus=abc_corpus,
    freqs_corpus=freqs_corpus_trigramm,
    n_gram=3,
    n_iters=10000,
    n_trials=20,
)

Best likelihood: -1725.6602016838426
Accept raito: 0.044895


In [1770]:
apply_mapping(message, best_reverse_mapping)

'если вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать скорее всего вы все сделали правильно и получите максимальный балл за последнее четвертое задание курса хотя конечно я ничего не обещаш'

In [1771]:
original_message = "если вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать скорее всего вы все сделали правильно и получите максимальный балл за последнее четвертое задание курса хотя конечно я ничего не обещаю"
encoded_message = apply_mapping(message, best_reverse_mapping)
char_accuracy(original_message, encoded_message)

0.9956521739130435

Получилось почти идеально, качество выросло. 

**2. Попробуем тексты подлиннее, может что-то пойдет не так**

In [1778]:
mapping = generate_mapping(corpus_freqs)
encoded_text = apply_mapping(tokenized_text, mapping)
text_freqs = get_freqs(encoded_text)

corpus_freqs_sorted = sorted(corpus_freqs.items(), key=lambda x: x[1], reverse=True)
text_freqs_sorted = sorted(text_freqs.items(), key=lambda x: x[1], reverse=True)

abc_corpus = ''.join([c for c, _ in corpus_freqs_sorted])
abc_text = ''.join([c for c, _ in text_freqs_sorted])
abc_corpus, abc_text

(' оеанитслвркдмупяьгыбзчжйшхюэцщфъ', 'гуъпеюиофбтцр ыядшьскжзмлэвйчанхщ')

In [1785]:
best_reverse_mapping, _ = get_reverse_mapping_mcmc(
    encoded_text, 
    abc_encoded=abc_text, 
    abc_corpus=abc_corpus,
    freqs_corpus=freqs_corpus,
    n_gram=2,
    n_iters=10000,
    n_trials=5,
)
print('-----')
print(encoded_text)
print('-----')
decoded_text = apply_mapping(encoded_text, best_reverse_mapping)
print(decoded_text)
print('-----')
print(char_accuracy(tokenized_text, decoded_text))

Best likelihood: -4991.84417224061
Accept raito: 0.00686
-----
фгопьпъипгцъузилгьюеуфгйпбтурглутцегеижптгеузъ фйиесгъюжгеопртшъъумгябудибрумгфгруоубумгфюбитуесгьоуоугъюгбпжруеосгфуъэьппгзутуфюгпзугдытюгуя апъюгъюгзб жсгигуъгрюкютешгцъпгяулувицгъюгеобюъъ эгоуфн эгяоин гего ертыцигепбыцигяпбсшцигигьпбъыцглулутруцгиоюрг уоеуъгерюкютгуъгфъпкюяъугфыгъпгеудибюпопесгфртюжыфюосгефуигедпбпвпъишгфгэвъуюхбирюъерипгнпъъыпгд цюзигшгфкжбузъ тгуог жифтпъишгрюргъигябифыргшгргъпудыьюмъыцгеяуеудъуеошцглутцеюгчоугфъпкюяъупгфоубвпъипгфгеюцыпгоюмъыпгцуигцыетигдытугеуфпбйпъъугъпудщшеъицыцгрюргьпбогфуксцигфыгудгчоуцг къютигеябуеитгшгуъгяуфпбъ тешгъюгео тпгжпбвюгфгб рпгжыцша эешгябудибр гигпзугзт дуругеижшаипгзтюкюгбюжуеоъугкюдтиеоютигябикъюмопесг уоеуъгьоугфыгеуфпбйпъъугедиоыгегоутр герюкютгуъгябикъюэесгцъпгетпжуфютугдыгкюеоюфиосгфюегъюяиеюосгудгчоуцгъюгтиеоуьрпгд цюзигигяужяиеюосешгяуьпц гяуоуц гьоугьпбпкгяшосгциъ огфыгерювпопгьоугфепгчоугъпудыьюмъугябуеоу
-----
в течение многих часов шерлок холмс сидел сог

In [1786]:
best_reverse_mapping, _ = get_reverse_mapping_mcmc(
    encoded_text, 
    abc_encoded=abc_text, 
    abc_corpus=abc_corpus,
    freqs_corpus=freqs_corpus_trigramm,
    n_gram=3,
    n_iters=10000,
    n_trials=5,
)

print('-----')
print(encoded_text)
print('-----')
decoded_text = apply_mapping(encoded_text, best_reverse_mapping)
print(decoded_text)
print('-----')
print(char_accuracy(tokenized_text, decoded_text))

Best likelihood: -6859.4582829474975
Accept raito: 0.00766
-----
фгопьпъипгцъузилгьюеуфгйпбтурглутцегеижптгеузъ фйиесгъюжгеопртшъъумгябудибрумгфгруоубумгфюбитуесгьоуоугъюгбпжруеосгфуъэьппгзутуфюгпзугдытюгуя апъюгъюгзб жсгигуъгрюкютешгцъпгяулувицгъюгеобюъъ эгоуфн эгяоин гего ертыцигепбыцигяпбсшцигигьпбъыцглулутруцгиоюрг уоеуъгерюкютгуъгфъпкюяъугфыгъпгеудибюпопесгфртюжыфюосгефуигедпбпвпъишгфгэвъуюхбирюъерипгнпъъыпгд цюзигшгфкжбузъ тгуог жифтпъишгрюргъигябифыргшгргъпудыьюмъыцгеяуеудъуеошцглутцеюгчоугфъпкюяъупгфоубвпъипгфгеюцыпгоюмъыпгцуигцыетигдытугеуфпбйпъъугъпудщшеъицыцгрюргьпбогфуксцигфыгудгчоуцг къютигеябуеитгшгуъгяуфпбъ тешгъюгео тпгжпбвюгфгб рпгжыцша эешгябудибр гигпзугзт дуругеижшаипгзтюкюгбюжуеоъугкюдтиеоютигябикъюмопесг уоеуъгьоугфыгеуфпбйпъъугедиоыгегоутр герюкютгуъгябикъюэесгцъпгетпжуфютугдыгкюеоюфиосгфюегъюяиеюосгудгчоуцгъюгтиеоуьрпгд цюзигигяужяиеюосешгяуьпц гяуоуц гьоугьпбпкгяшосгциъ огфыгерювпопгьоугфепгчоугъпудыьюмъугябуеоу
-----
в течение многих часов шерлок холмс сидел с

Качество практически не изменилось.

**3. Попробуем еще одну перестановку, но уже с очень коротким текстом**

In [1794]:
short_text = 'это очень короткий текст чтобы проверить гипотезу что совсем маленькие тексты раскодируются последовательностями из двух букв хуже чем из трех'

mapping = generate_mapping(corpus_freqs)
encoded_text = apply_mapping(short_text, mapping)
text_freqs = get_freqs(encoded_text)

corpus_freqs_sorted = sorted(corpus_freqs.items(), key=lambda x: x[1], reverse=True)
text_freqs_sorted = sorted(text_freqs.items(), key=lambda x: x[1], reverse=True)

abc_corpus = ''.join([c for c, _ in corpus_freqs_sorted])
abc_text = ''.join([c for c, _ in text_freqs_sorted])
abc_corpus, abc_text

(' оеанитслвркдмупяьгыбзчжйшхюэцщфъ', 'убефсымвогхпит шцьчзрнжйящюа')

**Биграммы**

In [1795]:
best_reverse_mapping, _ = get_reverse_mapping_mcmc(
    encoded_text, 
    abc_encoded=abc_text, 
    abc_corpus=abc_corpus,
    freqs_corpus=freqs_corpus,
    n_gram=2,
    n_iters=10000,
    n_trials=5,
)
print('-----')
print(encoded_text)
print('-----')
decoded_text = apply_mapping(encoded_text, best_reverse_mapping)
print(decoded_text)
print('-----')
print(char_accuracy(short_text, decoded_text))

Best likelihood: -795.9068114645025
Accept raito: 0.04672
-----
йбеуехфтпумевебмсяубфмыбухберну веофвсбпущс ебфшгухбеуыеоыфиуицьфтпмсфубфмыбнувцымечсвгюбыжу еыьфчеоцбфьптеыбжисусшучогзургмоузгафухфиусшубвфз
-----
щсо одела тоностих сетвс дсочь кнорениса эикосейы дсо ворвем мушелатие сетвсь нувтопиныъсвя ковшепорусешаловсями ий прыб чытр быюе дем ий снеб
-----
0.4154929577464789


**Триграммы**

In [1796]:
best_reverse_mapping, _ = get_reverse_mapping_mcmc(
    encoded_text, 
    abc_encoded=abc_text, 
    abc_corpus=abc_corpus,
    freqs_corpus=freqs_corpus_trigramm,
    n_gram=3,
    n_iters=10000,
    n_trials=5,
)

print('-----')
print(encoded_text)
print('-----')
decoded_text = apply_mapping(encoded_text, best_reverse_mapping)
print(decoded_text)
print('-----')
print(char_accuracy(short_text, decoded_text))

Best likelihood: -1126.9021005498848
Accept raito: 0.0466
-----
йбеуехфтпумевебмсяубфмыбухберну веофвсбпущс ебфшгухбеуыеоыфиуицьфтпмсфубфмыбнувцымечсвгюбыжу еыьфчеоцбфьптеыбжисусшучогзургмоузгафухфиусшубвфз
-----
это очерь новотний тенст чтому пводевить бипотеха что содсел лышерьние тенсту высногиваются посшегодытешьростяли их гдак манд каже чел их твек
-----
0.647887323943662


**4-граммы**

In [1797]:
freqs_corpus_fourgramm = get_freqs_smooth(tokenized_corpus, n_gram=4)

In [1798]:
best_reverse_mapping, _ = get_reverse_mapping_mcmc(
    encoded_text, 
    abc_encoded=abc_text, 
    abc_corpus=abc_corpus,
    freqs_corpus=freqs_corpus_fourgramm,
    n_gram=4,
    n_iters=10000,
    n_trials=5,
)

print('-----')
print(encoded_text)
print('-----')
decoded_text = apply_mapping(encoded_text, best_reverse_mapping)
print(decoded_text)
print('-----')
print(char_accuracy(short_text, decoded_text))

Best likelihood: -1412.4571300090768
Accept raito: 0.05224
-----
йбеуехфтпумевебмсяубфмыбухберну веофвсбпущс ебфшгухбеуыеоыфиуицьфтпмсфубфмыбнувцымечсвгюбыжу еыьфчеоцбфьптеыбжисусшучогзургмоузгафухфиусшубвфз
-----
это очень короткий текст чтобы проверить гипотезу что совсем маленькие тексты раскодируются последовательностями из двух букв хуже чем из трех
-----
1.0


Получилось просто идеально.

**Наблюдение по n-граммам**

Похоже на то, что триграммы дают хороший прирост качества для декодирования коротких текстов. Для длинных текстов в принципе и так все хорошо.

## 6.  Применения для этой модели

Бонус: какие вы можете придумать применения для этой модели? Пляшущие человечки ведь не так часто встречаются в жизни (хотя встречаются! и это самое потрясающее во всей этой истории, но об этом я расскажу потом).

Я пока что не в курсе конкретных задач биоинформатики, но подозреваю, что такие модели могут использоваться для работы с последовательностями ДНК. Там число вариаций просто огромно и явно больше чем число жителей земного шара, и, думаю, обычные методы здесь не подходят. Скорее всего, одно из применений - как раз в анализе последовательностей ДНК в биоинформатике.